In [1]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.fire_data import *
from src.data.read_data import *
from src.gen_functions import *
from src.features.dataset import Dataset
import seaborn as sns
output_notebook()
# set font size 
plt.rcParams.update({'font.size': 16})

Loading BokehJS ...

In [3]:
b_folder='../data/pm25/'
a4th_folder ='../data/air4thai_hourly/'
cm_folder ='../data/cm_proc/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'

In [3]:
KEY = os.environ.get('T_KEY')

# Function

In [28]:
def get_b_box(x_cen, y_cen, width):
    """Obtain the bounding box given the center coordinate and the step size.
    
    x_cen and y_cen must be in mercator coordinate
    
    Args:
        x_cen: Longitude center of the box 
        y_cen: Latitude center of the box 
        width: the width of the bounding box in km
    
    Return xy_arr
    
    """
    # define the width of the bounding box
 
    step = width*0.5
    
    return  [(x_cen+step, y_cen+step), (x_cen-step, y_cen+step), (x_cen+step, y_cen-step), (x_cen-step, y_cen-step) ]

In [5]:
def get_pts_on_ln(ln,line_types, num_pts=25):
    """Get points along a straight line. 
    
    The line can be horizontal or vertical lines
    
    Args:
        ln: a pair of lat,long coordinate that define a line 
        line_type: either vertical ('v_line') or horitzontal ('h_line')
        num_pts: number of points needed
        
    """
    line_type = 'v_line'
    if line_type == 'v_line':
        y = np.mean(ln[:,1])
        x = ln[:,0]
        x = np.linspace(x[0], x[1], num_pts).round(7)
        
        coor = [[o, y] for o in x]
        
    elif line_type == 'h_line':
        x = np.mean(ln[:,0])
        y = ln[:,1]
        y = np.linspace(y[0], y[1], num_pts).round(7)
        coor = [[x, o] for o in y]
        
    else:
        raise AssertionError('The line_type must be either v_line or h_line.')
                             
    return np.array(coor)
    
    

In [6]:
city_json = glob(b_folder + '*.json')[0]
with open(city_json) as f:
    cities_info = json.load(f)

In [9]:
bkk_info = cities_info[0]
cm_info = {'Country': 'Thailand',
 'City': 'Chiang Mai',
 'City (ASCII)': 'Chiang Mai',
 'Region': 'Chiang Mai',
 'Region (ASCII)': 'Chiang Mai',
 'Population': '200952',
 'Latitude': '18.7904',
 'Longitude': '98.9847',
 'Time Zone': 'Asia/Bangkok'}

x_lon = cm_info['Longitude']
y_lat = cm_info['Latitude']

x_cen = merc_x(x_lon)
y_cen = merc_y(y_lat)

# define the width of the bounding box
width = 5E3
xy_arr = get_b_box(x_cen, y_cen, width)
v_ln1 = np.array([xy_arr[1], xy_arr[2]])
v_ln2 = np.array([xy_arr[0], xy_arr[3]])

h_ln1 = np.array([xy_arr[0], xy_arr[1]])
h_ln2 = np.array([xy_arr[2], xy_arr[3]])

stepx = 1E4
stepy = stepx

# range bounds supplied in web mercator coordinates
p = figure(x_range=(x_cen-1*stepx,x_cen+1*stepx), y_range=(y_cen-stepy*1, y_cen+stepy*1),
           x_axis_type="mercator", y_axis_type="mercator")

p.add_tile(get_provider(Vendors.STAMEN_TERRAIN_RETINA))


p.circle(x_cen,y_cen,color='red',line_color='black',size=10, legend_label='Chiang Mai')
p.line(v_ln1[:,0], v_ln1[:,1],color='blue', line_width=2)
p.line(v_ln2[:,0], v_ln2[:,1],color='blue', line_width=2)
p.line(h_ln1[:,0], h_ln1[:,1],color='red', line_width=2)
p.line(h_ln2[:,0], h_ln2[:,1],color='red', line_width=2)

# obtain coor_list 
num_pts = 5
coor_arr = np.vstack([get_pts_on_ln(ln, ln_type, num_pts) for ln, ln_type in zip(line_list, line_type)])
p.circle( np.apply_along_axis(merc_x, arr=coor_arr[ :2*num_pts,1], axis=0),[merc_y(y) for y in coor_arr[ :2*num_pts, 0 ]], color='red' )


show(p)

In [8]:
xy_arr = get_b_box(x_cen, y_cen, 5E3)

# convert to lat-lon 
xy_latlon = np.apply_along_axis(to_latlon, arr=xy_arr, axis=1 )
# get line pairs 
v_ln1 = np.array([xy_latlon[1], xy_latlon[2]])
v_ln2 = np.array([xy_latlon[0], xy_latlon[3]])

h_ln1 = np.array([xy_latlon[0], xy_latlon[1]])
h_ln2 = np.array([xy_latlon[2], xy_latlon[3]])

# find points long each lines 
line_type = ['v_line','v_line', 'h_line', 'h_line']
line_list = [v_ln1, v_ln2, h_ln1, h_ln2]
num_pts = 10
coor_list = np.vstack([get_pts_on_ln(ln, ln_type, num_pts) for ln, ln_type in zip(line_list, line_type)])
# add center coordinate 
coor_list = np.concatenate(([[y_lat , x_lon ]], coor_list))

# convert to points for REST API
pts_list = coor_list.astype(str)
pts_list = np.apply_along_axis(','.join, arr=pts_list, axis=1)
pts_list = np.apply_along_axis('|'.join, arr=pts_list, axis=0).tolist()

In [51]:
# obtain the nearest roads for center points
url = f'https://roads.googleapis.com/v1/nearestRoads?points={pts_list}&key={KEY}'
res = requests.get(url)
# extract json
res = res.json()['snappedPoints']
# convert to dataframe 
res_df = pd.DataFrame(res)
res_df = pd.concat([res_df, pd.json_normalize(res_df['location'])], axis=1)

# extract place ID
c_place = res_df[res_df['originalIndex']==0].iloc[0]['placeId']
# drop original 
place_list = res_df[res_df['originalIndex']!=0]['placeId'].dropna().unique()

In [61]:
org_pl = c_place
des_pl = place_list[1]

In [62]:
dp_time = 1343641500

In [63]:
org_pl

'ChIJiWnCLZo62jAROsJFeDBcCSM'

In [64]:
des_pl

'ChIJN3Hlfls62jARSZlQh4DGu88'

In [67]:
url = f'https://maps.googleapis.com/maps/api/directions/json?origin=place_id:{org_pl}&destination=place_id:{des_pl}&departure_time={dp_time}&key={KEY}'
#url = f'https://maps.googleapis.com/maps/api/directions/json?origin=place_id:{org_pl}&destination=place_id:{des_pl}&key={KEY}'

res = requests.get(url).json()

In [68]:
res

{'error_message': 'departure_time is in the past. Traffic information is only available for future and current times.',
 'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJiWnCLZo62jAROsJFeDBcCSM',
   'types': ['route']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJN3Hlfls62jARSZlQh4DGu88',
   'types': ['route']}],
 'routes': [],
 'status': 'INVALID_REQUEST'}

In [49]:
res['routes'][0]['legs'][0]

{'distance': {'text': '5.6 km', 'value': 5644},
 'duration': {'text': '11 mins', 'value': 684},
 'end_address': 'Unnamed Road, Tambon Si Phum, Amphoe Mueang Chiang Mai, Chang Wat Chiang Mai 50200, Thailand',
 'end_location': {'lat': 18.7902368, 'lng': 98.98468009999999},
 'start_address': 'Khuang Sing Rd, Tambon Chang Phueak, Amphoe Mueang Chiang Mai, Chang Wat Chiang Mai 50300, Thailand',
 'start_location': {'lat': 18.8118716, 'lng': 98.9797048},
 'steps': [{'distance': {'text': '69 m', 'value': 69},
   'duration': {'text': '1 min', 'value': 22},
   'end_location': {'lat': 18.8118421, 'lng': 98.9803592},
   'html_instructions': 'Head <b>east</b> on <b>ถนน ข่วงสิงห์</b> toward <b>Anusawaree Singha S. 3</b>',
   'polyline': {'points': 'eeyqBc~b{Q?W@a@?MB{@'},
   'start_location': {'lat': 18.8118716, 'lng': 98.9797048},
   'travel_mode': 'DRIVING'},
  {'distance': {'text': '23 m', 'value': 23},
   'duration': {'text': '1 min', 'value': 13},
   'end_location': {'lat': 18.8116334, 'lng': 9

In [7]:
city_json = glob(b_folder + '*.json')[0]
with open(city_json) as f:
    cities_info = json.load(f)
    
city_info = cities_info[0]


cm_info = {'Country': 'Thailand',
 'City': 'Chiang Mai',
 'City (ASCII)': 'Chiang Mai',
 'Region': 'Chiang Mai',
 'Region (ASCII)': 'Chiang Mai',
 'Population': '200952',
 'Latitude': '18.7904',
 'Longitude': '98.9847',
 'Time Zone': 'Asia/Bangkok'}

In [16]:
city_info

{'Country': 'Viet Nam',
 'City': 'Bÿc Giang',
 'City (ASCII)': 'Bac Giang',
 'Region': 'Bÿc Giang',
 'Region (ASCII)': 'Bac Giang',
 'Population': '53728',
 'Latitude': '21.2731',
 'Longitude': '106.1946',
 'Time Zone': 'Asia/Ho_Chi_Minh'}

In [26]:
city_info = cm_info

x_lon = city_info['Longitude']
y_lat = city_info['Latitude']

x_cen = merc_x(x_lon)
y_cen = merc_y(y_lat)

# define the width of the bounding box
width = 1E3
xy_arr = get_b_box(x_cen, y_cen, width)
# convert to (lat, long)
p0 = to_latlon(xy_arr[0])
p1 = to_latlon(xy_arr[-1])

In [24]:
to_latlon((x_cen, y_cen))

array([ 21.2731, 106.1946])

In [27]:
(p1[1].round(4), p1[0].round(4), p0[1].round(4), p0[0].round(4) )

(98.9802, 18.7861, 98.9892, 18.7947)

In [21]:
(p0[0].round(4), p0[1].round(4), p1[0].round(4), p1[1].round(4) )

(21.2773, 106.1991, 21.2689, 106.1901)

In [2]:
dataset = Dataset('Chiang Mai')
dataset.load_()

In [58]:
row

latitude      19.8946
longitude     105.114
confidence         52
lat_km           2246
long_km         11701
distance      694.093
power           20.52
count               1
country       Vietnam
Name: 2002-07-04 13:04:00, dtype: object

In [80]:
row = dataset.fire.iloc[2000]

# convert to mercator
x_cen = merc_x(row['longitude'])
y_cen = merc_y(row['latitude'])
# define the width of the bounding box
width = 1E3


xy_arr = get_b_box(x_cen, y_cen, width)
# convert to (lat, long)
p0 = to_latlon(xy_arr[0])
p1 = to_latlon(xy_arr[-1])

url = f'http://www.overpass-api.de/api/xapi?*[landuse=*][bbox={p1[1].round(4)},{p1[0].round(4)},{p0[1].round(4)},{p0[0].round(4) }]'
print(url)

http://www.overpass-api.de/api/xapi?*[landuse=*][bbox=100.0529,15.0588,100.0619,15.0676]


In [81]:
# request for land use information
res = requests.get(url)
# convert to Element Tree
root = ET.fromstring(res.text)
# convert to dictionary
res_json = etree_to_dict(root)

res_json['osm']

{'note': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.',
 'meta': {'@osm_base': '2020-09-27T17:19:02Z'},
 '@version': '0.6',
 '@generator': 'Overpass API 0.7.56.6 474850e8'}

In [11]:
import xml.etree.ElementTree as ET

In [3]:
res = requests.get('http://www.overpass-api.de/api/xapi?*[landuse=*][bbox=10.52412,52.27387,10.52635,52.27505]')

{'note': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.',
 'meta': {'@osm_base': '2020-09-27T17:16:03Z'},
 '@version': '0.6',
 '@generator': 'Overpass API 0.7.56.6 474850e8'}

In [53]:
def return_tag(d):
    
    if type(d) is dict:
        for k in d.keys():
            print(k)
            if k =='tag':
                return d[k]
            else:
                yield return_tag(d[k])
        
    

In [55]:
for result in return_tag(res_json['osm']):
    print(result)

note
<generator object return_tag at 0x000001EEE0DC7548>
meta
<generator object return_tag at 0x000001EEE0DC7648>
node
<generator object return_tag at 0x000001EEE0DC76C8>
way
<generator object return_tag at 0x000001EEE0DC7648>
@version
<generator object return_tag at 0x000001EEE0DC76C8>
@generator
<generator object return_tag at 0x000001EEE0DC7648>


In [23]:
from collections import defaultdict

def etree_to_dict(t)->dict:
    """Convert XML root to dictionary
    
    https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary
        
    """
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [21]:
etree_to_dict(root)

{'osm': {'note': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.',
  'meta': {'@osm_base': '2020-09-27T16:44:02Z'},
  'node': [{'@id': '256777112', '@lat': '52.2767634', '@lon': '10.5180260'},
   {'@id': '300286165', '@lat': '52.2751073', '@lon': '10.5246400'},
   {'@id': '300286166', '@lat': '52.2749894', '@lon': '10.5197503'},
   {'@id': '300286169', '@lat': '52.2758705', '@lon': '10.5185683'},
   {'@id': '300286527', '@lat': '52.2767596', '@lon': '10.5186417'},
   {'@id': '300286528', '@lat': '52.2765568', '@lon': '10.5169229'},
   {'@id': '300286532', '@lat': '52.2759752', '@lon': '10.5153374'},
   {'@id': '300286533', '@lat': '52.2759473', '@lon': '10.5133626'},
   {'@id': '300286534', '@lat': '52.2753936', '@lon': '10.5131307'},
   {'@id': '300286535', '@lat': '52.2748482', '@lon': '10.5133345'},
   {'@id': '300287756', '@lat': '52.2737430', '@lon': '10.5138246'},
   {'@id': '300287758', '@lat': '52.2734672', '@lon': '10.5